## Gradient Boosting Hyperparameter Tuning (F1-Score)

In [2]:
# Load Libraries

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score

from sklearn.ensemble import GradientBoostingClassifier

import optuna

In [4]:
# Load Data

df = pd.read_csv('bank_4.csv', index_col=0)

In [6]:
# Train / Test Split

X = df.drop(columns=['churn', 'complain', 'umap_1', 'umap_2'])
y = df['churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [8]:
# Hyperparameter tuning

def objective(trial):

    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 1.0),
        'max_depth': trial.suggest_int('max_depth', 2, 50),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 32),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 32)
    }

    model = GradientBoostingClassifier(
        **params,
        random_state=42)
    
    threshold = 0.33
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    scores = []
    
    for tr, te in skf.split(X_train, y_train):
        
        X_tr, X_te = X_train.iloc[tr], X_train.iloc[te]
        y_tr, y_te = y_train.iloc[tr], y_train.iloc[te]
        
        model.fit(X_tr, y_tr)
        prob = model.predict_proba(X_te)[:, 1]
        y_pred = np.where(prob < threshold, 0, 1)
        
        scores.append(f1_score(y_te, y_pred))
        
    return np.mean(scores)

In [10]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, show_progress_bar=True)

[I 2024-07-10 00:01:19,050] A new study created in memory with name: no-name-2b2a2301-8cea-4f93-828e-08d74710e431


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2024-07-10 00:01:54,157] Trial 0 finished with value: 0.5366305587213784 and parameters: {'n_estimators': 1639, 'learning_rate': 0.367705706952336, 'max_depth': 32, 'min_samples_split': 12, 'min_samples_leaf': 19}. Best is trial 0 with value: 0.5366305587213784.
[I 2024-07-10 00:02:22,923] Trial 1 finished with value: 0.557995303023943 and parameters: {'n_estimators': 338, 'learning_rate': 0.16763520307195676, 'max_depth': 40, 'min_samples_split': 27, 'min_samples_leaf': 30}. Best is trial 1 with value: 0.557995303023943.
[I 2024-07-10 00:02:38,266] Trial 2 finished with value: 0.5396415033428086 and parameters: {'n_estimators': 1242, 'learning_rate': 0.8174722023994216, 'max_depth': 37, 'min_samples_split': 31, 'min_samples_leaf': 12}. Best is trial 1 with value: 0.557995303023943.
[I 2024-07-10 00:04:51,262] Trial 3 finished with value: 0.58279844277694 and parameters: {'n_estimators': 1601, 'learning_rate': 0.011135832122879077, 'max_depth': 15, 'min_samples_split': 31, 'min_samp

In [12]:
print("Best trial:", study.best_trial)
print("Best hyperparameters:", study.best_params)

Best trial: FrozenTrial(number=61, state=1, values=[0.6156758664009493], datetime_start=datetime.datetime(2024, 7, 10, 1, 25, 38, 901409), datetime_complete=datetime.datetime(2024, 7, 10, 1, 26, 30, 492582), params={'n_estimators': 1601, 'learning_rate': 0.007484423302277796, 'max_depth': 4, 'min_samples_split': 29, 'min_samples_leaf': 10}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=2000, log=False, low=100, step=1), 'learning_rate': FloatDistribution(high=1.0, log=False, low=0.001, step=None), 'max_depth': IntDistribution(high=50, log=False, low=2, step=1), 'min_samples_split': IntDistribution(high=32, log=False, low=2, step=1), 'min_samples_leaf': IntDistribution(high=32, log=False, low=1, step=1)}, trial_id=61, value=None)
Best hyperparameters: {'n_estimators': 1601, 'learning_rate': 0.007484423302277796, 'max_depth': 4, 'min_samples_split': 29, 'min_samples_leaf': 10}
